# Applied Data Science Capstone Week 3 - Assignment:Segmenting and Clustering Neighborhoods in Toronto

In [58]:
# First, we import the libraries:
import requests
import lxml.html as lh
import pandas as pd

In [42]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# To manage de contents of the webpage, we will create page:
page = requests.get(url)

# We will save the contents of the website:
doc = lh.fromstring(page.content)

# Now we parse the data:
tr_elements = doc.xpath('//tr')

# We check the length of the first 12 rows:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [43]:
# We will use the first row as header:
tr_elements = doc.xpath('//tr')

#For each row we will save the header in an empty list:
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [44]:
# The data is stored after the header:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    #If row is not of size 3, we do not want that data:
    if len(T)!=3:
        break
    
    i=0
    
    # We check each element:
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert values to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list:
        col[i][1].append(data)
        i+=1
        
# We now check the length of each column, the should all be equal:
[len(C) for (title,C) in col]

[287, 287, 287]

In [55]:
# We create the dataframe:

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Access the top 5 rows of the data frame 
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [62]:
# We order the dataframe:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

# We delete the '\n':
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Downtown Toronto,Adelaide,M5H
1,Scarborough,Agincourt,M1S
2,Scarborough,Agincourt North,M1V
3,Etobicoke,Albion Gardens,M9V
4,Etobicoke,Alderwood,M8W


In [59]:
# We delete all the 'Not assigned' boroughs:

df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

# We join similar Postcodes and boroughs:

df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,Adelaide,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough
2,Agincourt North,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke
5,Bathurst Manor,M3H,North York
6,Bathurst Quay,M5V,Downtown Toronto
7,Bayview Village,M2K,North York
8,Beaumond Heights,M9V,Etobicoke
9,Bedford Park,M5M,North York


In [60]:
# We trim spaces at the start of the strings:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#If Neighbourhood is 'Not assigned' we will assign 'Borough':
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood


In [61]:
# Check the shape of the data frame
df.shape

(287, 3)